In [1]:
import pandas as pd
import os

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df.head()

,Days_Since_Start,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Volume,TSLA_RSI_7,TSLA_RSI_14,TSLA_MACD,TSLA_MACD_Signal,...,NEE_MACD_Signal,HD_Open,HD_High,HD_Low,HD_Close,HD_Volume,HD_RSI_7,HD_RSI_14,HD_MACD,HD_MACD_Signal
0,196,1.424667,1.483333,1.394667,1.460667,37297500.0,92.220196,43.408794,-0.064685,-0.083391,...,0.059857,19.278965,19.349948,19.058918,19.215080,17061600,25.229445,25.954283,-0.802271,-0.824908
1,197,1.456667,1.456667,1.336667,1.353333,27379500.0,71.260949,39.206237,-0.058209,-0.078355,...,0.071355,18.987938,19.896520,18.895661,19.825537,23279400,44.368562,43.190715,-0.756203,-0.811167
2,198,1.377333,1.393333,1.300000,1.348000,18787500.0,74.198423,39.159154,-0.052897,-0.073263,...,0.079636,19.719066,19.882326,19.413839,19.499018,17161300,37.425292,44.047680,-0.737539,-0.796441
3,199,1.366667,1.416667,1.358000,1.400000,14367000.0,72.916658,46.915168,-0.043984,-0.067407,...,0.087182,19.669378,20.208849,19.669378,20.031391,26142700,44.414890,52.659520,-0.672043,-0.771562
4,200,1.412667,1.437333,1.404000,1.419333,9804000.0,65.010249,57.983173,-0.034957,-0.060917,...,0.093594,19.974605,20.130768,19.804246,20.052687,18979200,49.562788,54.446405,-0.611372,-0.739524


In [2]:
import pandas as pd

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Days_Since_Start'])
df.set_index('Days_Since_Start', inplace=True)

# 确保其他列是数值类型，去掉可能的字符串类型列（例如日期）
df = df.apply(pd.to_numeric, errors='coerce')

# 检查数据的头部
print(df.head())

                  TSLA_Open  TSLA_High  TSLA_Low  TSLA_Close  TSLA_Volume  \
Days_Since_Start                                                            
196                1.424667   1.483333  1.394667    1.460667   37297500.0   
197                1.456667   1.456667  1.336667    1.353333   27379500.0   
198                1.377333   1.393333  1.300000    1.348000   18787500.0   
199                1.366667   1.416667  1.358000    1.400000   14367000.0   
200                1.412667   1.437333  1.404000    1.419333    9804000.0   

                  TSLA_RSI_7  TSLA_RSI_14  TSLA_MACD  TSLA_MACD_Signal  \
Days_Since_Start                                                         
196                92.220196    43.408794  -0.064685         -0.083391   
197                71.260949    39.206237  -0.058209         -0.078355   
198                74.198423    39.159154  -0.052897         -0.073263   
199                72.916658    46.915168  -0.043984         -0.067407   
200             

In [3]:
rows, cols = df.shape
print(f"DataFrame 共有 {rows} 行, {cols} 列")


DataFrame 共有 2633 行, 91 列


In [4]:
# 检查数据的列名
print(df.columns)

# 检查是否有缺失值
print(df.isnull().sum())


Index(['TSLA_Open', 'TSLA_High', 'TSLA_Low', 'TSLA_Close', 'TSLA_Volume',
       'TSLA_RSI_7', 'TSLA_RSI_14', 'TSLA_MACD', 'TSLA_MACD_Signal',
       'AAPL_Open', 'AAPL_High', 'AAPL_Low', 'AAPL_Close', 'AAPL_Volume',
       'AAPL_RSI_7', 'AAPL_RSI_14', 'AAPL_MACD', 'AAPL_MACD_Signal', 'GE_Open',
       'GE_High', 'GE_Low', 'GE_Close', 'GE_Volume', 'GE_RSI_7', 'GE_RSI_14',
       'GE_MACD', 'GE_MACD_Signal', 'QQQ_Open', 'QQQ_High', 'QQQ_Low',
       'QQQ_Close', 'QQQ_Volume', 'QQQ_RSI_7', 'QQQ_RSI_14', 'QQQ_MACD',
       'QQQ_MACD_Signal', 'NVDA_Open', 'NVDA_High', 'NVDA_Low', 'NVDA_Close',
       'NVDA_Volume', 'NVDA_RSI_7', 'NVDA_RSI_14', 'NVDA_MACD',
       'NVDA_MACD_Signal', 'UNH_Open', 'UNH_High', 'UNH_Low', 'UNH_Close',
       'UNH_Volume', 'UNH_RSI_7', 'UNH_RSI_14', 'UNH_MACD', 'UNH_MACD_Signal',
       'CAT_Open', 'CAT_High', 'CAT_Low', 'CAT_Close', 'CAT_Volume',
       'CAT_RSI_7', 'CAT_RSI_14', 'CAT_MACD', 'CAT_MACD_Signal', 'AMZN_Open',
       'AMZN_High', 'AMZN_Low', 'AMZN_

In [5]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd

class TradingEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}
    
    def __init__(self, df, window_size=30, initial_balance=10000, max_drawdown=0.6, risk_free_rate=0.001, transaction_cost=0.00005):
        super(TradingEnv, self).__init__()
        
        # 交易环境的初始参数
        self.df = df  # 交易数据
        self.window_size = window_size  # 观察窗口大小
        self.initial_balance = initial_balance  # 初始账户余额
        self.max_drawdown = max_drawdown  # 最大回撤限制
        self.risk_free_rate = risk_free_rate  # 无风险利率
        self.transaction_cost = transaction_cost  # 交易成本（降低到 0.005%）
        
        # 提取所有股票代码
        self.tickers = sorted(set(col.split('_')[0] for col in df.columns if '_' in col and not col.startswith("Days")))
        self.num_stocks = len(self.tickers)  # 股票数量
        self.features_per_stock = 9  # 每只股票的特征数
        
        # 定义观察空间（90 维度 = 30 天 * 3 支股票 * 9 个特征）
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(window_size * 90,), dtype=np.float32)
        
        # 定义动作空间（每只股票 [-0.5, 0.5]，表示买卖的目标资金比例）
        self.action_space = spaces.Box(low=-0.5, high=0.5, shape=(self.num_stocks,), dtype=np.float32)

        self.reset()
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # 初始化环境状态
        self.current_step = self.window_size  # 当前时间步（避免未定义的观察窗口）
        self.balance = self.initial_balance  # 账户余额
        self.shares_held = np.zeros(self.num_stocks)  # 持仓
        self.total_profit = 0  # 总盈利
        self.peak_value = self.initial_balance  # 账户最高市值
        self.done = False  # 终止标志
        self.history = [self.initial_balance]  # 账户历史市值
        
        return self._next_observation(), {}
    
    def _next_observation(self):
        # 获取过去 window_size 天的特征数据，展平后作为观察状态
        feature_cols = [col for col in self.df.columns if col != "Days_Since_Start"]
        obs = self.df.iloc[self.current_step - self.window_size:self.current_step][feature_cols].values.flatten()
        return obs

    def step(self, action):
        if self.done:
            return self._next_observation(), 0, True, False, {}

        # 获取当前时间步的收盘价
        close_prices = self.df.iloc[self.current_step, [self.df.columns.get_loc(f"{t}_Close") for t in self.tickers]]
        
        # 计算当前总资产（现金 + 股票持仓价值）
        total_value = self.balance + np.dot(self.shares_held, close_prices.values)
        
        # 计算目标投资金额（基于动作）
        target_values = np.clip(action, -0.5, 0.5) * total_value
        target_shares = target_values / (close_prices.values + 1e-6)
        delta_shares = target_shares - self.shares_held
        
        total_cost = 0  # 记录总交易成本
        for i in range(self.num_stocks):
            current_price = close_prices.iloc[i]
            cost = abs(delta_shares[i]) * current_price * self.transaction_cost
            total_cost += cost

            if delta_shares[i] > 0:  # 买入
                max_shares = self.balance // (current_price + cost)
                buy_shares = min(max_shares, delta_shares[i])
                self.shares_held[i] += buy_shares
                self.balance -= buy_shares * current_price + cost

            elif delta_shares[i] < 0:  # 卖出
                sell_shares = min(abs(delta_shares[i]), self.shares_held[i])
                self.shares_held[i] -= sell_shares
                self.balance += sell_shares * current_price - cost

        # 计算新的总资产
        total_value = self.balance + np.dot(self.shares_held, close_prices.values)
        prev_total_value = self.history[-1] if len(self.history) > 0 else self.initial_balance
        percentage_return = (total_value - prev_total_value) / (prev_total_value + 1e-6)

        self.total_profit = total_value - self.initial_balance
        self.peak_value = max(self.peak_value, total_value)
        drawdown = (self.peak_value - total_value) / (self.peak_value + 1e-6)

        # 计算夏普比率（过去 30 天）
        self.history.append(total_value)
        sharpe_ratio = 0  
        if len(self.history) > 30:
            returns = np.diff(self.history[-30:]) / (np.array(self.history[-30:-1]) + 1e-3)
            if len(returns) > 1 and np.std(returns) > 1e-3:
                sharpe_ratio = (np.mean(returns) - self.risk_free_rate) / (np.std(returns) + 1e-3) * np.sqrt(252)

        # 计算奖励函数
        reward = percentage_return * 100  # 主要奖励：收益率
        reward += 5 * max(0, (total_value - self.peak_value) / (self.peak_value + 1e-6))  # 奖励创新高
        reward -= drawdown * 2  # 惩罚回撤
        reward += max(sharpe_ratio, -2) * 0.1  # 奖励高夏普比率
        reward -= total_cost * 5  # 惩罚交易成本
        reward += np.sum(abs(delta_shares)) * 0.01  # 轻微鼓励交易

        if drawdown >= self.max_drawdown:
            reward -= 2  # 严格惩罚超出最大回撤

        self.current_step += 1
        self.done = self.current_step >= len(self.df) - 1  # 是否到达数据末尾

        return self._next_observation(), reward, self.done, False, {}
    
    def render(self, mode="human"):
        print(f'Step: {self.current_step}, Balance: {self.balance:.2f}, Shares: {self.shares_held}, Profit: {self.total_profit:.2f}')

print("TradingEnv ready!")



TradingEnv ready!


In [6]:
import gymnasium as gym
import numpy as np

# 假设 df 是你已经加载并预处理好的 DataFrame，包含所有需要的特征
# df = pd.read_csv('your_stock_data.csv')

# 创建环境实例
env = TradingEnv(df)  # 使用你的数据

# 测试环境初始化
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)

# 测试 reset 和 _next_observation
obs, info = env.reset()
print("Initial Observation:", obs)


Observation Space: Box(-inf, inf, (2700,), float32)
Action Space: Box(-0.5, 0.5, (10,), float32)
Initial Observation: [ 1.42466700e+00  1.48333299e+00  1.39466703e+00 ... -7.21857963e-02
 -1.47767060e-01  2.35000000e+02]


In [7]:
# 测试执行一步操作
action = np.random.uniform(-1, 1, size=(env.num_stocks,))  # 随机生成动作
print("Test Action:", action)

# 进行一步操作并获取返回的观察值、奖励等
obs, reward, done, truncated, info = env.step(action)
print("New Observation:", obs)
print("Reward:", reward)
print("Done:", done)


Test Action: [ 0.21716638  0.14054943 -0.55246393 -0.91145496 -0.17527138 -0.40037147
 -0.03342553 -0.24198969  0.87309856  0.62982447]
New Observation: [ 1.45666695e+00  1.45666695e+00  1.33666694e+00 ... -8.11938459e-02
 -1.34452417e-01  2.38000000e+02]
Reward: 60.148444109634404
Done: False


In [8]:
# 测试多步操作
for i in range(10):
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Step {i+1}, Reward: {reward}, Total Profit: {env.total_profit}, Done: {done}")
    if done:
        break


Step 1, Reward: 153.8613470449102, Total Profit: -92.52737688195339, Done: False
Step 2, Reward: 275.7427363882779, Total Profit: 295.72280657671945, Done: False
Step 3, Reward: 277.8562133723837, Total Profit: 421.58913608150215, Done: False
Step 4, Reward: 262.6204574287415, Total Profit: 565.7832500222612, Done: False
Step 5, Reward: 100.404059428204, Total Profit: 486.7632554536067, Done: False
Step 6, Reward: 333.3120795728501, Total Profit: 666.5893656522585, Done: False
Step 7, Reward: 187.84345368038692, Total Profit: 696.0559242068084, Done: False
Step 8, Reward: 182.26273745479213, Total Profit: 709.8681198652994, Done: False
Step 9, Reward: 270.1920624713515, Total Profit: 764.4645440870936, Done: False
Step 10, Reward: 272.71402707390007, Total Profit: 783.7238568135745, Done: False


In [9]:
# 测试环境是否能正常结束
env.reset()
while not env.done:
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Balance: {env.balance}, Total Profit: {env.total_profit}")


Balance: 0.808946302937529, Total Profit: -2.094822660992577
Balance: 609.3431650565972, Total Profit: 80.36677609416802
Balance: 5597.643195409832, Total Profit: 361.84339154814734
Balance: 29.726635929733675, Total Profit: 448.40289792204567
Balance: 11.012807766664313, Total Profit: 584.454135828968
Balance: 2498.8734851597255, Total Profit: 559.0460759411835
Balance: 4.577779589600062, Total Profit: 598.7799406405338
Balance: 2397.6202144066, Total Profit: 568.4782811466321
Balance: 5023.354878417557, Total Profit: 461.4278442832874
Balance: 25.655040303800206, Total Profit: 527.6814093812318
Balance: 0.969822970354816, Total Profit: 557.84387700361
Balance: 724.6713612977609, Total Profit: 549.3352648612708
Balance: 28.02871308556155, Total Profit: 495.7844144771261
Balance: 6781.446336174692, Total Profit: 527.0407510644727
Balance: 4029.8823150859284, Total Profit: 587.4815116476693
Balance: 6.946574279400352, Total Profit: 564.1969619093798
Balance: 204.29162480711568, Total Pr

In [10]:
import torch
import torch.nn as nn
import gymnasium as gym
import numpy as np
import pandas as pd
from stable_baselines3 import SAC
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

# 自定义 LSTM+Attention 特征提取器
class LSTMAttentionFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, lstm_hidden_size=64, attention_size=32):
        super(LSTMAttentionFeatureExtractor, self).__init__(observation_space, features_dim=lstm_hidden_size)
        
        self.input_dim = observation_space.shape[0]  # shape=(window_size * num_features,)
        self.window_size = 30  # 在 TradingEnv 中设置的窗口大小
        self.num_features = self.input_dim // self.window_size  # 每个时间步的特征数
        self.lstm_hidden_size = lstm_hidden_size
        self.attention_size = attention_size

        # LSTM 层
        self.lstm = nn.LSTM(input_size=self.num_features, hidden_size=self.lstm_hidden_size, batch_first=True)

        # 注意力层
        self.attention = nn.Linear(self.lstm_hidden_size, self.attention_size)
        self.attn_weights = nn.Linear(self.attention_size, 1, bias=False)

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        batch_size = observations.shape[0]
        # 调整输入形状 (batch, window_size, num_features)
        lstm_input = observations.view(batch_size, self.window_size, self.num_features)
        # LSTM 前向传播
        lstm_out, _ = self.lstm(lstm_input)
        # 注意力计算
        attn_scores = torch.tanh(self.attention(lstm_out))  
        attn_weights = torch.softmax(self.attn_weights(attn_scores), dim=1)  
        context_vector = torch.sum(attn_weights * lstm_out, dim=1)  
        return context_vector

# 定义调度函数
def lr_schedule(progress_remaining):
    """
    学习率调度函数
    progress_remaining: 1.0 (开始) 到 0.0 (结束)
    从 0.0003 线性衰减到 0.0001
    """
    return 0.0001 + 0.0002 * progress_remaining

def ent_coef_schedule(progress_remaining):
    """
    熵系数调度函数
    progress_remaining: 1.0 到 0.0
    从 0.1 线性衰减到 0.01
    """
    return 0.01 + 0.09 * progress_remaining

# 读取数据
file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

# 创建环境（假设 TradingEnv 已经定义好）
env = TradingEnv(df)
env = Monitor(env)  # 使用 Monitor 包装
env = DummyVecEnv([lambda: env])

# 定义自定义 LSTM+Attention 特征提取器的 SAC policy
policy_kwargs = dict(
    features_extractor_class=LSTMAttentionFeatureExtractor,
    features_extractor_kwargs=dict(lstm_hidden_size=64, attention_size=32),
    net_arch=[256, 256]  # 隐藏层大小
)

# 创建 SAC 模型，同时传入学习率和熵系数调度函数
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    policy_kwargs=policy_kwargs,
    buffer_size=200000,
    learning_rate=lr_schedule,  # 学习率使用调度函数
    batch_size=1024,
    gamma=0.99,
    tau=0.005,
    ent_coef="auto",  # 使用自动调节熵系数模式
    target_entropy=-np.prod(env.action_space.shape).astype(np.float32),  # 设定目标熵，可根据实际情况调整
    train_freq=64,
    gradient_steps=64
)


# 训练模型
model.learn(total_timesteps=1000000)  # 可根据需要增加训练步数

# 保存模型
model.save("sac_lstm_attention_trained_model")

# 评估模型
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")



Using cuda device
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 2.6e+03   |
|    ep_rew_mean     | 4.1e+05   |
| time/              |           |
|    episodes        | 4         |
|    fps             | 15        |
|    time_elapsed    | 688       |
|    total_timesteps | 10408     |
| train/             |           |
|    actor_loss      | -5.89e+03 |
|    critic_loss     | 2.15e+04  |
|    ent_coef        | 1.94      |
|    ent_coef_loss   | 0.0505    |
|    learning_rate   | 0.000298  |
|    n_updates       | 10304     |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 2.6e+03   |
|    ep_rew_mean     | 4.79e+05  |
| time/              |           |
|    episodes        | 8         |
|    fps             | 14        |
|    time_elapsed    | 1421      |
|    total_timesteps | 20816     |
| train/             |           |
|    actor_loss      | -1.06e+04 |
| 

KeyboardInterrupt: 